# **Chains**

chains are a fundamental concept that allows you to execute complex tasks in a structured and efficient way.

## **Why Chains?**

Chains are invaluable due to their capacity to effortlessly blend diverse components, shaping a singular and coherent application. Through the creation of chains, multiple elements can seamlessly come together. Imagine this scenario: a chain is crafted to take in user input, polish it using a PromptTemplate, and subsequently pass on this refined response to a large language model (LLM). This streamlined process not only simplifies but also enriches the overall functionality of the system. In essence, chains serve as the linchpin, seamlessly connecting different parts of the application and enhancing its capabilities.


In [ ]:
%%capture
# update or install the necessary libraries
!pip install --upgrade langchain langchain_community langchain_aws python-dotenv

In [ ]:
!pip uninstall -y numpy pandas
!pip install --no-cache-dir numpy==1.26.4 pandas

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_DEFAULT_REGION"] = os.getenv("AWS_DEFAULT_REGION")

In [ ]:
import pandas as pd
df = pd.read_csv("./content/employee.csv")

In [ ]:
df.head()

# **LLM Chain** - **The simplest chain**

The LLMChain is a foundational system that includes a PromptTemplate, an OpenAI model (such as a Large Language Model or a ChatModel), and optionally, an output parser. It operates by transforming input parameters using the PromptTemplate into a coherent prompt, which is then fed into the model. The resulting output is further refined and formatted into a usable form by the OutputParser, if provided. This structured approach ensures effective utilization of language models for various applications, enhancing their functionality and utility.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    temperature=0.5
)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What are the tools that need to be learned to earn the {designation}?,provide me one tool"
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
designation = "Devops Engineer"
chain.invoke(designation)

# **SimpleSequentialChain**

Simple Sequential Chains allow for a single input to undergo a series of coherent transformations, resulting in a refined output. This sequential approach ensures systematic and efficient handling of data, making it ideal for scenarios where a linear flow of information processing is essential

In [ ]:
# SimpleSequentialChain
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What are the tools that need to be learned to earn the {designation}?,provide me one tool"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    tool:{tool_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.invoke(designation)

# **SequentialChain**

A sequential chain is a chain that combines various individual chains, where the output of one chain serves as the input for the next in a continuous sequence. It operates by running a series of chains consecutively.

In [ ]:
# SequentialChain
from langchain.chains import SequentialChain

In [ ]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following designation"
    "\n\n{Designation}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="Designation_description"
                    )


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following description in 1 sentence:"
    "\n\n{Designation_description}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )

In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What are the promgramming language that need to be learned for this designation in one line:\n\n{Designation}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="programming_language"
                      )

In [ ]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a short follow up response to the following "
    "Write a short summary about the programming language:"
    "\n\nSummary: {summary}\n\nLanguage: {programming_language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [ ]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Designation"],
    output_variables=["Designation_description","programming_language", "summary","followup_message"],
    verbose=True
)

In [ ]:
designation = df.Designation[2]
overall_chain.invoke(designation)

# **Router Chain**

The Router Chain is used for complicated tasks. If we have multiple subchains, each of which is specialized for a particular type of input, we could have a router chain that decides which subchain to pass the input to.

In [ ]:
software_engineer_template = """You are a highly skilled software engineer. \
You excel at addressing queries about programming and software development in a clear
and straightforward manner. \
When faced with a challenge outside your expertise, you candidly \
acknowledge the gap in your knowledge. \

Here is a question:
{input}"""


database_administrator_template = """You are a highly skilled database administrator.\
You excel at answering questions about databases in a clear and precise manner, \
making even complex topics accessible to anyone.\
When faced with a query or issue you're unfamiliar with, \
you candidly admit that you need to look into it further, valuing accuracy over conjecture.\

Here is a question:
{input}"""

data_scientist_template = """You are a highly skilled data scientist.\
You excel at interpreting complex datasets and presenting insights in a \
straightforward and comprehensible manner.\
When confronted with a query beyond your expertise, you acknowledge \
your limitations and advocate for further research or consultation.

Here is a question:
{input}"""


Machine_learning_engineer_template = """ You are a highly skilled Machine Learning Engineer.\
You excel at answering questions about machine learning algorithms and models in a clear and succinct manner.\
When confronted with a query outside your expertise, you candidly admit that you don't have the answer. \

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "software_engineer",
        "description": "Good for answering questions about software_engineer",
        "prompt_template": software_engineer_template
    },
    {
        "name": "database_administrator",
        "description": "Good for answering database_administrator questions",
        "prompt_template": database_administrator_template
    },
    {
        "name": "data_scientist",
        "description": "Good for answering data_scientist questions",
        "prompt_template": data_scientist_template
    },
    {
        "name": "Machine_learning_engineer",
        "description": "Good for answering Machine_learning_engineer questions",
        "prompt_template": Machine_learning_engineer_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = r"""Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising \
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not \
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>
"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.invoke("What are the various categories of software?")

In [ ]:
chain.invoke("What are the differences between supervised and unsupervised learning")

# **Let's Do an Activity**

## **Objective**

Practice using chains with language models and structured output parsing to handle complex tasks efficiently.

## **Scenario**

You are tasked with building a system to assist users in understanding various technical concepts related to software engineering roles. Your goal is to use different chains to process user queries, transform them using prompt templates, and extract specific information using output parsers.

## **Steps**

* Define a Prompt Template
* Prepare Sample Queries
* Implement LLM Chains
* Output Parsing
* Interactive Chain Execution
* Evaluate